In [2]:
!pip install --upgrade numpy
!pip install librosa
!pip install tensorflow==2.11
!pip install scipy
!pip install scikit-learn
!pip install keras

  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Users\\ASUS\\anaconda3\\Lib\\site-packages\\~5mpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
pyldavis 3.4.0 requires numpy>=1.22.0, but you have numpy 1.21.6 which is incompatible.


In [6]:
from pydub import AudioSegment
from pydub.playback import play
import random
import os

# Set the path to the directory containing the audio files
audio_folder = "D:/voicedatset"

# Loop through all the audio files in the folder
for file_name in os.listdir(audio_folder):
    if file_name.endswith(".wav"):
        # Load the audio file
        audio_file = AudioSegment.from_file(os.path.join(audio_folder, file_name), format="wav")
        
        # Apply audio augmentation
        # Example 1: Change the playback speed
        speed_change = audio_file.speedup(playback_speed=random.uniform(0.8, 1.2))
        speed_change.export(os.path.join(audio_folder, "speed_" + file_name), format="wav")
        
        # Example 2: Apply volume change
        volume_change = audio_file + random.randint(-10, 10)
        volume_change.export(os.path.join(audio_folder, "volume_" + file_name), format="wav")
        
        # Example 3: Apply pitch change
        pitch_change = audio_file + random.randint(-5, 5)
        pitch_change.export(os.path.join(audio_folder, "pitch_" + file_name), format="wav")


In [5]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set the data directory
data_dir = "D:/voicedatset/train"

# Define the classes and the corresponding labels
classes = {"lightoff": 0, "lighton":1, "fanon":2, "fanoff":3 }


# Define the parameters for feature extraction
sampling_rate = 16000
duration = 1
n_mfcc = 20

# Define a function to extract features and labels from the audio files
def extract_features(file_path, n_frames=100):
    signal, sr = librosa.load(file_path, sr=sampling_rate, duration=duration)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
    mfccs = pad_sequences([mfccs.T], maxlen=n_frames, padding='post', truncating='post')
    return mfccs[0].flatten()

# Iterate through the data directory to extract the features and labels
X = []
y = []
for class_name, label in classes.items():
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        features = extract_features(file_path)
        
        X.append(features)
        y.append(label)

# Convert the data to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and validation sets
np.random.seed()
tf.random.set_seed(42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Convert the labels to one-hot encoded vectors
y_train = to_categorical(y_train, num_classes=len(classes))
y_val = to_categorical(y_val, num_classes=len(classes))

# Define the model architecture
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    #layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    #layers.Dropout(0.3),
    layers.Dense(len(classes), activation='softmax')
])


# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=40,
                    validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_dir = "D:/voicedatset/test"
X_test = []
y_test = []
for class_name, label in classes.items():
    class_dir = os.path.join(test_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        features = extract_features(file_path)
        X_test.append(features)
        y_test.append(label)
X_test = np.array(X_test)
print("x test ", X_test)
y_test = np.array(y_test)
y_test = to_categorical(y_test)
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
# Save the trained model
model.save("my_model.h5")


Epoch 1/50
4/4 [==============================] - 1s 59ms/step - loss: 30.2847 - accuracy: 0.3377 - val_loss: 24.9484 - val_accuracy: 0.3889
Epoch 2/50
4/4 [==============================] - 0s 16ms/step - loss: 11.8935 - accuracy: 0.5714 - val_loss: 6.4893 - val_accuracy: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 19ms/step - loss: 6.3838 - accuracy: 0.5714 - val_loss: 7.8036 - val_accuracy: 0.5000
Epoch 4/50
4/4 [==============================] - 0s 17ms/step - loss: 1.7927 - accuracy: 0.7792 - val_loss: 6.2893 - val_accuracy: 0.4444
Epoch 5/50
4/4 [==============================] - 0s 17ms/step - loss: 1.1998 - accuracy: 0.8247 - val_loss: 3.7340 - val_accuracy: 0.7222
Epoch 6/50
4/4 [==============================] - 0s 16ms/step - loss: 0.4962 - accuracy: 0.9026 - val_loss: 4.4575 - val_accuracy: 0.6667
Epoch 7/50
4/4 [==============================] - 0s 19ms/step - loss: 0.2273 - accuracy: 0.9351 - val_loss: 3.4463 - val_accuracy: 0.7222
Epoch 8/50
4/4 [========